### User Intent 1: Load the datasets into pandas DataFrames

In [ ]:

import os
import pandas as pd
import re
import math
import copy
import numpy as np
import sys
# Add the root directory /workspaces/llm_etl to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))
# Now import your module
from spider2_utils import load_csv_database
# Load environment variables

-setup-

In [ ]:
_database = load_csv_database("education_business", rows_limit=-1)

# Assume CSV files or other sources are available for reading
sales_df = _database["hardware_fact_sales_monthly"]
product_df = _database["hardware_dim_product"]


### Question
Can you provide a list of hardware product segments along with their unique product counts for 2020 in the output, ordered by the highest percentage increase in unique fact sales products from 2020 to 2021?

### User Intent 1: Filter sales data for the year 2020

In [ ]:

sales_2020 = sales_df[sales_df['fiscal_year'] == 2020]


### User Intent 2: Join filtered 2020 sales data with product information

In [ ]:

sales_2020_joined = pd.merge(sales_2020, product_df, on='product_code', how='inner')


### User Intent 3: Group by segment and count unique products for 2020

In [ ]:

unique_2020 = sales_2020_joined.groupby('segment')['product_code'].nunique().reset_index()
unique_2020.rename(columns={'product_code': 'unique_products_2020'}, inplace=True)


### User Intent 4: Filter sales data for the year 2021

In [ ]:

sales_2021 = sales_df[sales_df['fiscal_year'] == 2021]


### User Intent 5: Join filtered 2021 sales data with product information

In [ ]:

sales_2021_joined = pd.merge(sales_2021, product_df, on='product_code', how='inner')


### User Intent 6: Group by segment and count unique products for 2021

In [ ]:

unique_2021 = sales_2021_joined.groupby('segment')['product_code'].nunique().reset_index()
unique_2021.rename(columns={'product_code': 'unique_products_2021'}, inplace=True)


### User Intent 7: Merge 2020 and 2021 unique product counts on segment

In [ ]:

merged = pd.merge(unique_2020, unique_2021, on='segment')


### User Intent 8: Calculate percentage change and sort by it in descending order

In [ ]:

merged['percentage_change'] = ((merged['unique_products_2021'] - merged['unique_products_2020']) * 100.0) / merged['unique_products_2020']
merged['product_count_2020'] = merged['unique_products_2020']
result = merged[['segment', 'product_count_2020', 'percentage_change']].copy()
result = result.sort_values(by='percentage_change', ascending=False).reset_index(drop=True)
result
